In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from numpy import load

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

In [ ]:
#train dataset

X_train_face=[]
X_train_iris=[]
y_train_face=[]
y_train_iris=[]


os.chdir('/content/drive/MyDrive/생체인증보안/04_multimodal_training')
for i in os.listdir():
    img = cv2.imread(i)   
    img = cv2.resize(img,(160,160))
    if(i[4:8]=="face"):
        X_train_face.append(img)
        y_train_face.append(i[0:3])
    if(i[4:8]=="iris"):
        X_train_iris.append(img)
        y_train_iris.append(i[0:3])

In [ ]:
#test dataset

X_test_face = []
X_test_iris = []
y_test = []

os.chdir('/content/drive/MyDrive/생체인증보안/04_multimodal_test')
for i in os.listdir():
    img = cv2.imread(i)   
    img = cv2.resize(img,(160,160))
    if(i[3:7]=="face" or i[2:6]=="face"):
        X_test_face.append(img)
    if(i[3:7]=="iris" or i[2:6]=="iris"):
        X_test_iris.append(img)
        y_test.append(i.rstrip("_iris.png"))

In [ ]:
#val dataset

X_val_face=[]
X_val_iris=[]
y_val=[]

os.chdir('/content/drive/MyDrive/생체인증보안/04_multimodal_training')
for i in os.listdir():
    img = cv2.imread(i)   
    img = cv2.resize(img,(160,160))
    if(i[4:8]=="face"):
        X_val_face.append(img)
    if(i[4:8]=="iris"):
        X_val_iris.append(img)
        y_val.append(i[0:3])

In [ ]:
print ("Shape of an image in X_train_face: ", X_train_face[0].shape)
print ("Shape of an image in X_train_iris: ", X_train_iris[0].shape)

print ("Shape of an image in X_train_face: ", X_val_face[0].shape)
print ("Shape of an image in X_train_iris: ", X_val_iris[0].shape)

print ("Shape of an image in X_train_face: ", X_test_face[0].shape)
print ("Shape of an image in X_train_iris: ", X_test_iris[0].shape)

print("Total categories: ", len(np.unique(y_train_iris)))
print("Total categories: ", len(np.unique(y_train_face)))
print("Total categories: ", len(np.unique(y_val)))
print("Total categories: ", len(np.unique(y_test)))

In [ ]:
le = preprocessing.LabelEncoder()

y_train_iris = le.fit_transform(y_train_iris)
y_train_iris = tf.keras.utils.to_categorical(y_train_iris, num_classes=64)
y_train_iris = np.array(y_train_iris)
y_train_face = le.fit_transform(y_train_face)
y_train_face = tf.keras.utils.to_categorical(y_train_face, num_classes=64)
y_train_face = np.array(y_train_face)

y_val = le.fit_transform(y_val)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=64)
y_val = np.array(y_val)

X_train_iris = np.array(X_train_iris)
X_test_iris = np.array(X_test_iris)
X_val_iris = np.array(X_val_iris)
X_train_face = np.array(X_train_face)
X_test_face = np.array(X_test_face)
X_val_face = np.array(X_val_face)

In [ ]:
from numpy import asarray
print("X_train_iris Shape: ", X_train_iris.shape)
print("X_train_face Shape: ", X_train_face.shape)
print("y_train_iris Shape: ", y_train_iris.shape)
print("y_train_face Shape: ", y_train_face.shape)

print("X_val_iris Shape: ", X_val_iris.shape)
print("X_val_face Shape: ", X_val_face.shape)
print("y_val Shape: ", y_val.shape)

print("X_test_iris Shape: ", X_test_iris.shape)
print("X_test_face Shape: ", X_test_face.shape)

In [ ]:
from imgaug import augmenters as iaa

seq = iaa.Sequential([
    # blur images with a sigma of 0 to 0.5
    iaa.GaussianBlur(sigma=(0, 0.5)),
    iaa.Affine(
        # scale images to 90-110% of their size, individually per axis
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        # translate by -10 to +10 percent (per axis)
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        # if mode is constant, use a cval between 0 and 255
        cval=255,
    )
], random_order=True)

x=X_train_iris
y=y_train_iris
aug=20

for i in range(aug):
	x_aug=seq.augment_images(x)
	X_train_iris=np.append(X_train_iris, x_aug, axis=0)
	y_train_iris=np.append(y_train_iris, y, axis=0)
 

x=X_train_face
y=y_train_face
aug=20

for i in range(aug):
	x_aug=seq.augment_images(x)
	X_train_face=np.append(X_train_face, x_aug, axis=0)
	y_train_face=np.append(y_train_face, y, axis=0)

In [ ]:
print("X_train_iris Shape: ", X_train_iris.shape)
print("X_train_face Shape: ", X_train_face.shape)
print("y_train_iris Shape: ", y_train_iris.shape)
print("y_train_face Shape: ", y_train_face.shape)

print("X_val_iris Shape: ", X_val_iris.shape)
print("X_val_face Shape: ", X_val_face.shape)
print("y_val Shape: ", y_val.shape)

print("X_test_iris Shape: ", X_test_iris.shape)
print("X_test_face Shape: ", X_test_face.shape)

In [ ]:
from keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [ ]:
################### 홍채 모델 ###################

from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers import LeakyReLU


# 기존 코드 수정
def get_iris_model():
    m1=Sequential()
    m1.add(BatchNormalization(input_shape = (160,160,3)))
    m1.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (160, 160, 3))) 
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Dropout(0.1))

    m1.add(Convolution2D(filters=64,kernel_size=3,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Dropout(0.1))

    m1.add(Convolution2D(filters=128,kernel_size=3,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Dropout(0.1))

    m1.add(Convolution2D(filters=256,kernel_size=3,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))

    m1.add(Flatten()) 
    m1.add(Dense(units=160,activation = 'relu'))
    m1.add(BatchNormalization())
    m1.add(Dense(units = 64, activation = 'softmax'))
    m1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', precision, recall, f1score])  
    return m1

In [ ]:
iris_model=get_iris_model()
iris_model.summary()
iris_model.fit(X_train_iris, y_train_iris, epochs=50, validation_data=(X_val_iris,y_val))

In [ ]:
################### 얼굴 모델 ###################

from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers import LeakyReLU

def get_face_model():
    m1=Sequential()
    m1.add(BatchNormalization(input_shape = (160,160,3)))
    m1.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (160, 160, 3))) 
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Convolution2D(filters=6,kernel_size=4,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Convolution2D(filters=160,kernel_size=3,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Convolution2D(filters=160,kernel_size=2,padding='same',activation='relu'))
    m1.add(MaxPooling2D(pool_size=2))
    m1.add(Flatten()) 
    m1.add(Dense(units=160,activation = 'relu'))
    m1.add(Dense(units = 64, activation = 'softmax'))
    m1.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics=['accuracy', precision, recall, f1score])
    return m1

In [ ]:
face_model=get_face_model()
face_model.summary()
face_model.fit(X_train_face, y_train_face, epochs=50, validation_data=(X_val_face,y_val))

In [ ]:
predicted_iris = iris_model.predict(X_test_iris, batch_size=128)
predicted_iris = np.argmax(np.round(predicted_iris),axis=1)
predicted_iris = predicted_iris.tolist()

print("Prediction :",predicted_iris)
print(y_test)

In [ ]:
predicted_face = face_model.predict(X_test_face, batch_size=128)
predicted_face = np.argmax(np.round(predicted_face),axis=1)
predicted_face = predicted_face.tolist()

print("Prediction :",predicted_face)
print(y_test)

In [ ]:
y_res_iris=[]
y_res_iris = map(int, y_test)

y_res_face=[]
y_res_face = map(int, y_test)

In [ ]:
result_iris = { name:value for name, value in zip(y_res_iris, predicted_iris) }
result_iris = sorted(result_iris.items())

print(result_iris)

result_face = { name:value for name, value in zip(y_res_face, predicted_face) }
result_face = sorted(result_face.items())

print(result_face)

In [ ]:
import csv
with open('1871080_이혜승_멀티모달_1차_답안_모델2개.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(result_iris)